In [1]:
import math

In [2]:
# An atomic term is a bunch of variables multiplied together, with no addition
# Example: c*c*x*y
class atomic_term:

    def __init__(self, input):
        self.components = input # List of the variables being multiplied 

    def __str__(self):

        out = ""
        
        for comp in self.components:
            out += str(comp)
            out += "*"

        return out[0:-1]


    # Gets variables 
    def variables(self):
        return [v for v in self.components if (isinstance(v, str) and v != "i")]
    
    # Gets variables and i's
    def nonConstants(self):
        return [v for v in self.components if isinstance(v, str)]
    
    # Gets the product of all constants
    def constants(self):
        return math.prod([c for c in self.components if isinstance(c, int)])
    

    
    # Checks if this term and input term have the same list of variables
    # This is used to simplify sums of atomic terms
    def like(self, inpTerm):

        # Get sorted list of non constants
        selfVars = sorted(self.nonConstants())
        inpVars = sorted(inpTerm.nonConstants())

        if len(selfVars) != len(inpVars):
            return False
        

        for i in range(len(selfVars)):
            if selfVars[i] != inpVars[i]:
                return False
            
        return True



    # Combines all i's and constant terms then sorts
    # Puts this term in a standard form 
    def clean(self):
        
        # New list of components to be filled
        newComponents = []

        # Get all constants and multiply them
        newConstant = self.constants()

        # If newConstant is zero make the entire term 0
        if newConstant == 0:
            self.components = [0]
            return

        # Count the number of i's
        iCount = len([im for im in self.components if im == "i"])

        # Multiply the constants with the i's
        if (iCount % 4 == 2 or iCount % 4 == 3):
            newConstant = -1 * newConstant 

        # Get the variables and sort them
        variables = [v for v in self.components if (isinstance(v, str) and v != "i")]
        variables.sort()

        # Add the variables into newComponents
        newComponents = variables
        
        # Add in i if necessary
        if (iCount%4 == 1 or iCount%4 == 3):
            newComponents.insert(0, "i")

        # Add the combined constants
        # If new constants is 1 and new components is not empty then ignore
        # ( Multiplying by 1 can be ignored )
        if (not newConstant == 1) or (len(newComponents) == 0):
            newComponents.insert(0, newConstant)

        self.components = newComponents

In [3]:
# A term sum is a sum of atomic terms
class term_sum:

    def __init__(self, terms):
        self.components = terms

    def __str__(self):
        out = ""

        for term in self.components:
            out += term.__str__()
            out += " + "
        
        return out[0:-3]

    def sort(self):
        self.components.sort()

    # Cleans all components and removes zero terms
    def clean(self):

        for term in self.components:
            term.clean()

        # Remove all zero terms
        zeroTerms = [term for term in self.components if len(term.components) == 1 and term.components[0] == 0]
        for term in zeroTerms:
            self.components.remove(term)


    # Combines terms which only differ by a (real) constant
    # Keeps terms which differ by i separate
    def combine_like_terms(self):

        # Necessary to combine all constants into one and account for i's
        self.clean()

        # List to be filled with new atomic terms
        newAtomicTerms = []

        # Repeatedly take this objects first term and combine it with all similar terms
        while len(self.components) != 0:
                
            # Find all atomic terms which have the same list of variables as the first term
            likeTerms = [term for term in self.components if self.components[0].like(term)]

            # Delte all these terms from self
            for term in likeTerms: self.components.remove(term)

            # Combine these terms into one and add to newAtomicTerms
            newConstant = sum([term.constants() for term in likeTerms])
            newAtomicTerms.append(atomic_term([newConstant] + likeTerms[0].nonConstants()))


        # Set the components of this object to be newAtomicTerms
        self.components = newAtomicTerms

In [7]:
# Multiplies two atomic terms, just combines the component lists. Does not clean.
def mulitply_atomic(term1, term2):
    return atomic_term(term1.components + term2.components)



# Multiplies two term_sums. Returns a term sum. Does not clean.
def multiply_out_terms(terms1, terms2):

    outTermSum = term_sum([])

    for term1 in terms1.components:
        for term2 in terms2.components:
            outTermSum.components.append(mulitply_atomic(term1, term2))

    return outTermSum


# Gets the imaginary and real components of a term_sum and returns them as term_sum's
def separate_parts(inpTermSum):

    inpTermSum.clean()

    realTerms = []
    imaginaryTerms = []

    for atomicTerm in inpTermSum.components:

        if "i" in atomicTerm.components:
            imaginaryTerms.append(atomic_term([var for var in atomicTerm.components if var != "i"]))
        else:
            realTerms.append(atomic_term([var for var in atomicTerm.components]))

    return (term_sum(realTerms), term_sum(imaginaryTerms))


In [8]:
replacements = []
replaceCount = 0

# Puts term_sum in format that can be put in code. Only enter in clean terms for convenience. 
# Does not separate real and imaginary parts, this must be done beforehand.
# Does not factor out variables in the nonFactor list
def format_term_sum(inpTermSum, noFactor):

    global replacements
    global replaceCount

    replacements = []
    replaceCount = 0

    return recursive_format_term_sum(inpTermSum, noFactor)


def recursive_format_term_sum(inpTermSum, noFactor):

    global replacements
    global replaceCount

    # Find the counts of each variable not in noFactor
    variableCounts = helper_variable_counts(inpTermSum, noFactor)


    # Base case: all terms are in nofactor so there is nothing to factor
    if len(variableCounts.values()) == 0:
        # Replace term with precalculated variable
        return helper_replace(inpTermSum)


    # Base case: No variable has a count over 1
    # This means that there are no terms that we could factor out to improve efficiency
    # If a term only appears once, factoring it out would turn
    # one multiplication into one multiplication
    if max(variableCounts.values()) <= 1:
        # Replace what can be replaced
        return helper_complex_replace(inpTermSum, noFactor)


    # Find the variable with the highest count
    # This is the one we will factor out
    variableToFactor = sorted(variableCounts.items(), key=lambda item: item[1], reverse=True)[0][0]


    # Factor out one copy of variableToFactor from each atomic term in inpTermSum
    nonFactoredTerms = term_sum([])
    factoredTerms = term_sum([])

    for atomicTerm in inpTermSum.components:

        if variableToFactor in atomicTerm.components:
            termCopy = atomicTerm.components.copy()
            termCopy.remove(variableToFactor)
            factoredTerms.components.append(
                atomic_term(
                    termCopy
                )
            )
        else:
            nonFactoredTerms.components.append(
                atomic_term(
                    atomicTerm.components.copy()
                )
            )


    # Check for empty nonFactoredTerms
    if len(nonFactoredTerms.components) == 0:
        return "(" + recursive_format_term_sum(factoredTerms, noFactor) + ") * " + variableToFactor
    else:
        return recursive_format_term_sum(nonFactoredTerms, noFactor) + " + (" + recursive_format_term_sum(factoredTerms, noFactor) + ") * " + variableToFactor



# Finds the 'counts' of each variable
# Each term containing one or more of a variables only counts as one instance of the variable
# Ignore the variables in noFactor
# e.x. a*a + a*b + c*2 counts 2 a's, 1 b, and 1 c
# This is used so that format_term_sum knows when to factor a variable out
def helper_variable_counts(inpTermSum, noFactor):
    
    variableSet = set()                             # Each unique variable
    for atomicTerm in inpTermSum.components:
        for term in atomicTerm.components:
            if (not isinstance(term, int)) and term not in noFactor:
                variableSet.add(term)

    variableDict = dict()                           # Stores counts of each variable
    for variable in variableSet:
        variableDict[variable] = 0


    # Count each distinct appearance of a variable 
    for atomicTerm in inpTermSum.components:

        for variable in variableSet:

            if variable in atomicTerm.components:
                variableDict[variable] += 1

    return variableDict


# Used when format_term_sum encounters a term that cannot be factored farther
# Stores code to precalculate input term and returns name of variable precalculated
# Input needs to only contain terms in noFactor
def helper_replace(inpTermSum):

    global replaceCount
    global replacements

    # Check if replacement has already been made
    for i in range(len(replacements)):
        if "float p" + str(i) + " = " + inpTermSum.__str__() + ";" == replacements[i]:
            # If so just return that variable name
            return "p" + str(i)

    replacements.append("float p" + str(replaceCount) + " = " + inpTermSum.__str__() + ";")
    toReturn = "p" + str(replaceCount)
    replaceCount += 1
    return toReturn


# helper_replace, but for term_sums with some amount of variables not in noFactor
# This is only run on terms where there is no point in factoring further
def helper_complex_replace(inpTermSum, noFactor):

    global replaceCount
    global replacements

    # List of atomic_terms to be put to together and returned as a string
    outTerms = []

    # Split atomic terms by whether or not they are constants
    constantTerms = []
    nonConstantTerms = []

    for atomicTerm in inpTermSum.components:

        constant = True
        
        for var in atomicTerm.components:
            if var not in noFactor:
                constant = False

        if constant: 
            constantTerms.append(atomicTerm)
        else:
            nonConstantTerms.append(atomicTerm)


    # Process constant terms
    constantTermAmount = sum([len(term.components) for term in constantTerms])

    # Only replace necessary terms, if there is only one constant there is no point in factoring
    if constantTermAmount > 1:
        outTerms.append(atomic_term([helper_replace(term_sum(constantTerms))]))
    elif constantTermAmount == 1:
        outTerms.append(atomic_term(constantTerms[0]))


    # Process nonconstant terms
    for term in nonConstantTerms:

        # Get the variables of term that are in/out of noFactor
        inNoFactor = [var for var in term.components if var in noFactor]
        outNoFactor = [var for var in term.components if var not in noFactor]
        
        # If there is one or less constant terms then do nothing
        if len(inNoFactor) <= 1:
            outTerms.append(term)
        else:
            # Replace constant terms
            outTerms.append(
                atomic_term(
                    [helper_replace(
                        term_sum([atomic_term(inNoFactor)])
                    )] 
                    + outNoFactor
                )
            )

    # Return the string representation of outTerms
    return term_sum(outTerms).__str__()



In [21]:
# Combine all of the previously defined functions to process a term sum
def to_code(terms, noFactor):

    # Combine like terms 
    terms.clean()
    terms.combine_like_terms()
    terms.clean()

    # Separate real and imaginary parts
    real_part, imag_part = separate_parts(terms)

    # Clean and format each part
    real_part.clean()
    real_eq = format_term_sum(real_part, noFactor)

    imag_part.clean()
    imag_eq = recursive_format_term_sum(imag_part, noFactor) # Use the recursive version to not reset replacements


    print(real_eq)
    print()
    print(imag_eq)
    print()
    for repl in replacements: print(repl)

    
    

In [20]:
# Build up polynomial for 52 parameter fractal
kernel_term_sum = term_sum([])

# Iterate through the different exponents for c, z, and zb
for cExp in range(3):
    for zExp in range(3):
        for zbExp in range(3):

            # Ignore the all zeros case
            if cExp + zExp + zbExp != 0:
                
                # Build up this term_sum by multiplying the correct amount of c', z's and zb's
                buildTerm = term_sum([atomic_term([1])])

                for i in range(cExp):
                    buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["cr"]),atomic_term(["i","ci"])])) # *(cr + i*ci)

                for i in range(zExp):
                    buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["zr"]),atomic_term(["i","zi"])])) # *(zr + i*zi)

                for i in range(zbExp):
                    buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["zr"]),atomic_term([-1, "i","zi"])])) # *(zr - i*zi)

                # Add the coefficient
                buildTerm = multiply_out_terms(buildTerm, term_sum([
                    atomic_term(["coeffs[" + str(2*cExp + 6*zExp + 18*zbExp - 2) + "]"]), 
                    atomic_term(["i", "coeffs[" + str(2*cExp + 6*zExp + 18*zbExp - 1) + "]"])
                    ]))

                # Add the components of buildTerm to kernel_term_sum
                for atomicTerm in buildTerm.components:
                    kernel_term_sum.components.append(atomic_term(atomicTerm.components))
                


# Process the polynomial
to_code(kernel_term_sum, ["cr", "ci"] + ["coeffs[" + str(i) + "]" for i in range(54)])

p0 + (p1 + (p2 + (p3 + (p4) * zr) * zr) * zr) * zr + (p5 + (p6 + (p7) * zr) * zr + (p8 + (p3 + (p9) * zr) * zr + (p7 + (p4) * zi) * zi) * zi) * zi

p10 + (p11 + (p12 + (p13 + (p14) * zr) * zr) * zr) * zr + (p15 + (p16 + (p17) * zr) * zr + (p18 + (p13 + (p19) * zr) * zr + (p17 + (p14) * zi) * zi) * zi) * zi

float p0 = coeffs[0]*cr + -1*ci*coeffs[1] + coeffs[2]*cr*cr + -2*ci*coeffs[3]*cr + -1*ci*ci*coeffs[2];
float p1 = coeffs[16] + coeffs[4] + coeffs[18]*cr + -1*ci*coeffs[19] + coeffs[6]*cr + -1*ci*coeffs[7] + coeffs[20]*cr*cr + -2*ci*coeffs[21]*cr + -1*ci*ci*coeffs[20] + coeffs[8]*cr*cr + -2*ci*coeffs[9]*cr + -1*ci*ci*coeffs[8];
float p2 = coeffs[34] + coeffs[22] + coeffs[10] + coeffs[36]*cr + -1*ci*coeffs[37] + coeffs[24]*cr + -1*ci*coeffs[25] + coeffs[12]*cr + -1*ci*coeffs[13] + coeffs[38]*cr*cr + -2*ci*coeffs[39]*cr + -1*ci*ci*coeffs[38] + coeffs[26]*cr*cr + -2*ci*coeffs[27]*cr + -1*ci*ci*coeffs[26] + coeffs[14]*cr*cr + -2*ci*coeffs[15]*cr + -1*ci*ci*coeffs[14];
float p3 = coeffs[4

In [18]:
# Build up a large term_sum that is the polynomial I need for the 52 parameter kernel
kernel_term_sum = term_sum([])

# Iterate through the different exponents for c, z, and zb
for cExp in range(3):
    for zExp in range(3):
        for zbExp in range(3):

            # Ignore the all zeros case
            if cExp + zExp + zbExp != 0:
                
                # Build up this term_sum by multiplying the correct amount of c', z's and zb's
                buildTerm = term_sum([atomic_term([1])])

                for i in range(cExp):
                    buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["cr"]),atomic_term(["i","ci"])])) # *(cr + i*ci)

                for i in range(zExp):
                    buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["zr"]),atomic_term(["i","zi"])])) # *(zr + i*zi)

                for i in range(zbExp):
                    buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["zr"]),atomic_term([-1, "i","zi"])])) # *(zr - i*zi)

                # Add the coefficient
                buildTerm = multiply_out_terms(buildTerm, term_sum([
                    atomic_term(["coeffs[" + str(2*cExp + 6*zExp + 18*zbExp - 2) + "]"]), 
                    atomic_term(["i", "coeffs[" + str(2*cExp + 6*zExp + 18*zbExp - 1) + "]"])
                    ]))

                # Add the components of buildTerm to kernel_term_sum
                for atomicTerm in buildTerm.components:
                    kernel_term_sum.components.append(atomic_term(atomicTerm.components))


In [19]:
to_code(kernel_term_sum, ["cr", "ci"] + ["coeffs[" + str(i) + "]" for i in range(54)])

p0 + (p1 + (p2 + (p3 + (p4) * zr) * zr) * zr) * zr + (p5 + (p6 + (p7) * zr) * zr + (p8 + (p3 + (p9) * zr) * zr + (p7 + (p4) * zi) * zi) * zi) * zi

p10 + (p11 + (p12 + (p13 + (p14) * zr) * zr) * zr) * zr + (p15 + (p16 + (p17) * zr) * zr + (p18 + (p13 + (p19) * zr) * zr + (p17 + (p14) * zi) * zi) * zi) * zi

float p0 = coeffs[0]*cr + -1*ci*coeffs[1] + coeffs[2]*cr*cr + -2*ci*coeffs[3]*cr + -1*ci*ci*coeffs[2];
float p1 = coeffs[16] + coeffs[4] + coeffs[18]*cr + -1*ci*coeffs[19] + coeffs[6]*cr + -1*ci*coeffs[7] + coeffs[20]*cr*cr + -2*ci*coeffs[21]*cr + -1*ci*ci*coeffs[20] + coeffs[8]*cr*cr + -2*ci*coeffs[9]*cr + -1*ci*ci*coeffs[8];
float p2 = coeffs[34] + coeffs[22] + coeffs[10] + coeffs[36]*cr + -1*ci*coeffs[37] + coeffs[24]*cr + -1*ci*coeffs[25] + coeffs[12]*cr + -1*ci*coeffs[13] + coeffs[38]*cr*cr + -2*ci*coeffs[39]*cr + -1*ci*ci*coeffs[38] + coeffs[26]*cr*cr + -2*ci*coeffs[27]*cr + -1*ci*ci*coeffs[26] + coeffs[14]*cr*cr + -2*ci*coeffs[15]*cr + -1*ci*ci*coeffs[14];
float p3 = coeffs[4

In [ ]:
buildTerm = term_sum([atomic_term([1])])

buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["cr"]),atomic_term(["i","ci"])])) # *(cr + i*ci)

buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["zr"]),atomic_term(["i","zi"])])) # *(zr + i*zi)

print(buildTerm)

1*cr*zr + 1*cr*i*zi + 1*i*ci*zr + 1*i*ci*i*zi


In [ ]:
kernel_term_sum.clean()
kernel_term_sum.combine_like_terms()
kernel_term_sum.clean()
print(kernel_term_sum)

coeffs[16]*zr + i*coeffs[17]*zr + -1*i*coeffs[16]*zi + coeffs[17]*zi + coeffs[34]*zr*zr + i*coeffs[35]*zr*zr + -2*i*coeffs[34]*zi*zr + 2*coeffs[35]*zi*zr + -1*coeffs[34]*zi*zi + -1*i*coeffs[35]*zi*zi + coeffs[4]*zr + i*coeffs[5]*zr + i*coeffs[4]*zi + -1*coeffs[5]*zi + coeffs[22]*zr*zr + i*coeffs[23]*zr*zr + coeffs[22]*zi*zi + i*coeffs[23]*zi*zi + coeffs[40]*zr*zr*zr + i*coeffs[41]*zr*zr*zr + -1*i*coeffs[40]*zi*zr*zr + coeffs[41]*zi*zr*zr + coeffs[40]*zi*zi*zr + i*coeffs[41]*zi*zi*zr + -1*i*coeffs[40]*zi*zi*zi + coeffs[41]*zi*zi*zi + coeffs[10]*zr*zr + i*coeffs[11]*zr*zr + 2*i*coeffs[10]*zi*zr + -2*coeffs[11]*zi*zr + -1*coeffs[10]*zi*zi + -1*i*coeffs[11]*zi*zi + coeffs[28]*zr*zr*zr + i*coeffs[29]*zr*zr*zr + i*coeffs[28]*zi*zr*zr + -1*coeffs[29]*zi*zr*zr + coeffs[28]*zi*zi*zr + i*coeffs[29]*zi*zi*zr + i*coeffs[28]*zi*zi*zi + -1*coeffs[29]*zi*zi*zi + coeffs[46]*zr*zr*zr*zr + i*coeffs[47]*zr*zr*zr*zr + 2*coeffs[46]*zi*zi*zr*zr + 2*i*coeffs[47]*zi*zi*zr*zr + coeffs[46]*zi*zi*zi*zi + i*coeff